<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Popular F1</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Popular
from lenskit.metrics.predict import rmse
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer():
    all_recs = []
    test_data = []
    pop = Popular(selector=None) # define algorithm

    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('Popular', pop, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    
    test_data = pd.concat(test_data, ignore_index=True)
    rla1 = topn.RecListAnalysis()
    rla2 = topn.RecListAnalysis()
    rla1.add_metric(topn.precision)
    rla2.add_metric(topn.recall)
    prec = rla1.compute(all_recs, test_data)
    rec = rla2.compute(all_recs, test_data)
    F1 = ((prec['precision'].values*rec['recall'].values*2)/(prec['precision'].values+rec['recall'].values))
    F1 = np.nan_to_num(F1)
    return [F1.mean(),prec['precision'].mean(),rec['recall'].mean()]

In [3]:
%%time
array = []
for i in range(1,100,1):
    array.append(model_trainer())

D:\Programas\Anaconda\Anaconda\Lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


Wall time: 9min 11s


In [5]:
metric = pd.DataFrame(array)
metric.columns = ['F1','Precision','Recall']
metric.head(5)

,F1,Precision,Recall
0,0.130553,0.087646,0.429051
1,0.131134,0.088134,0.429044
2,0.130214,0.087381,0.430953
3,0.128897,0.086744,0.418655
4,0.131433,0.088197,0.433305


In [7]:
metric.sort_values(by=['F1'],ascending=False).head(10)

,F1,Precision,Recall
94,0.132246,0.088802,0.430943
46,0.132053,0.088727,0.431350
92,0.131989,0.088759,0.427063
7,0.131981,0.088505,0.434293
74,0.131968,0.088579,0.432374
21,0.131813,0.088431,0.435442
52,0.131778,0.088261,0.433988
82,0.131770,0.088335,0.432145
16,0.131728,0.088399,0.432287
73,0.131664,0.088388,0.431685


In [8]:
metric.to_csv('metric_F1.csv', index=False)